# L16: Mutability in Lettuce

## Overview
* Explicit References
* New Eval
* Implicit References

## Tangent
* I re-discovered a quirk about Python that I'd like to share
    * I knew about this but I had forgotten
    * I came across it at work yesterday
* What does the following script print in Python3 / what would you expect it to print?
~~~
x = 2
def f():
    x = 4
f()
print(x)
~~~
<br /><br /><br /><br /><br /><br /><br /><br />

* It prints 2
* Why?
    * in python, a global variable is a local variable to the global scope
    * in python, a functions scope does not implicitly inherit the global scope

* What if I want it to print 4?
* There are two options here, namely:
    * use the global keywork in your function (best practice)
    * **heapify** the variable (sort of a hack)

* using global
~~~
x = 2
def f():
    global x
    x = 4
f()
print(x)
~~~

* heapifying x
~~~
x = [2]
def f():
    x[0] = 4
f()
print(x[0])
~~~

* just a fun fact / tangent
* You won't be tested on this
* You might consider such features if you were creating your own language with implicit references to mutable variables

* in a similar vein, the following python program throws an error
* it says, hey! you cant use x before it is declared
~~~
x = 2
def f():
    print(x)
    x = 4
f()
~~~

* Scala has a similar phenomina
* If we have an identifier in an outer scope we can't use it and then also shaddow the identifier

In [20]:
// Works
val x = 2
val y = {
    val x = 4
    x
}
println(y)

4


x: Int = 2
y: Int = 4

In [20]:
val a = 2
val b = {
    println(a)
    val a = 4
}

cmd20.sc:3: forward reference extends over definition of value a
    println(a)
            ^Compilation Failed

: 

## Explicit References in Lettuce
* We will now allow the Lettuce developer to be able to use mutable variables
* We will look at this with both:
    * Explicit References (like C)
    * Implicit References (like Python)
* For Explicit references we allow the programmer to:
    * create new reference cells with a value it it (NewRef)
    * lookup the value in the cell (DeRef)
    * change the value in the cell (AssignRef)

### Grammars - new stuff only
* NOTEs: 
    * here, the concrete lettuce syntax looks almost identical to the generative grammar for our AST
    * the references and addresses do not exist in the concrete syntax
    
#### Concrete Grammar
$$\begin{array} &
e & \rightarrow & NewRef(e) \\
& | & DeRef(e) \\
& | & AssignRef(e, e) \\
\end{array}$$
#### Generative Grammar
$$\begin{array} &
Expr & \rightarrow & NewRef(Expr) \\
& | & DeRef(Expr) \\
& | & AssignRef(Expr, Expr) \\
& | & Value \\
\\
Value & \rightarrow & Reference(Address) \\
\\
Address & \rightarrow & Int \\
\end{array}$$

### AST  - new stuff only

In [ ]:
sealed trait Expr
sealed trait Value extends Expr
type Address = Int

case class NewRef(e: Expr) extends Expr
case class DeRef(lval: Expr) extends Expr
case class AssignRef(lval: Expr, rval: Expr) extends Expr

sealed class Reference(a:Address) extends Value

### Examples - warming up, taking guesses

#### Easy Example
~~~
let x = NewRef(10) in 
let y = DeRef(x) + 1 in 
let z = AssignRef(x, y) in 
   DeRef(x)
~~~

#### Hard Example
~~~
let incr = function (x) 
           AssignRef(x, DeRef(x) + 1)
in 
let double = function (x) 
           AssignRef(x, DeRef(x) * 2)
in 
let r = NewRef(15) in 
let tmp0 = double(r) in 
let tmp1 = incr(r) in 
let tmp2 = double(r) in
  DeRef(r) 
~~~

### Visualizing memory
* For our purposes memory will be an immutable store
    * We can create a new stores as we go along
    * These new stores will look like the old store, but with incremental changes
* Let us visualize this as a table that grows
    * In real life, this is a table of fixed size and free space that is populated somewhat randomly over time
    * There is nothing truely random about that - usually
    * But the determinism of how space is allocated is quite complex and hardware dependent
    * So we won't worry about it in this course
    * Computer Engineering, OS, and Compilers would teach you more about this topic
* I begin with a empty table:
<table>
    <caption>Memory Size: 0</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
</table>

* Suppose I NewRef(15) I would then get:
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>15</td>
    </tr>
</table>

* Questions?

### Examples

#### Easy example
~~~
let x = NewRef(10) in 
let y = DeRef(x) + 1 in 
let z = AssignRef(x, y) in 
   DeRef(x)
~~~

##### 1
~~~
EVALUATE:
    let x = NewRef(10) in 
    let y = DeRef(x) + 1 in 
    let z = AssignRef(x, y) in 
       DeRef(x)
IN ENVIRONMENT:
    {}
AND STORE:
~~~
<table>
    <caption>Memory Size: 0</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
</table>

##### 2
* we have a let expression binding identifier 'x' to something
* evaluate the binding expression 'NewRef(10)' to value 'Reference(0)'
    * why 0? because the current store size is 0 and the size is uses as the new reference address
* map 'x' to the value of it's binding expression
* evaluate the body expression
~~~
EVALUATE:
    let y = DeRef(x) + 1 in 
    let z = AssignRef(x, y) in 
       DeRef(x)
IN ENVIRONMENT:
    {
    x -> Reference(0)
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>10</td>
    </tr>
</table>

##### 3
* we have a let expression binding identifier 'y' to something
* evaluate the binding expression 'Deref(x) + 1' to value '11'
    * in our environment , the identifier x = Reference(0) 
    * DeRef(x) = DeRef(Reference(0))
    * in our memory store, address 0 = 10
    * DeRef(Reference(0)) = 10
    * DeRef(x) + 1 = 10 + 1 = 11
* map 'y' to the value of it's binding expression '11'
* evaluate the body expression
~~~
EVALUATE:
    let z = AssignRef(x, y) in 
       DeRef(x)
IN ENVIRONMENT:
    {
    x -> Reference(0)
    y -> 11
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>10</td>
    </tr>
</table>

##### 4
* we have a let expression binding identifier 'z' to something
* evaluate the binding expression 'AssignRef(x, y)' to value '11'
    * in our environment , the identifier x = Reference(0) 
    * in our environment , the identifier y = 11
    * update the value at address 0 to value 11
    * the value of the whole expression is 11
* map 'z' to the value of it's binding expression '11'
* evaluate the body expression
~~~
EVALUATE:
   DeRef(x)
IN ENVIRONMENT:
    {
    x -> Reference(0)
    y -> 11
    z -> 11
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>11</td>
    </tr>
</table>

##### 4
* use envrionment to determine value of x: DeRef(x) = DeRef(Refernce(0))
* use the store to determine value at address 0: DeRef(Refernce(0)) = 11
* return 11
~~~
EVALUATE:
    11
IN ENVIRONMENT:
    {
    x -> Reference(0)
    y -> 11
    z -> 11
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>11</td>
    </tr>
</table>

##### 5
* We found a value, 11!

##### Qs????

#### Hard Example
~~~
let incr = function (x) 
           AssignRef(x, DeRef(x) + 1)
in 
let double = function (x) 
           AssignRef(x, DeRef(x) * 2)
in 
let r = NewRef(15) in 
let tmp0 = double(r) in 
let tmp1 = incr(r) in 
let tmp2 = double(r) in DeRef(r) 
~~~

##### 1
* evaluate the full expression
* environment starts as empty
* store starts as empty
~~~
EVALUATE:
    let incr = function (x) 
               AssignRef(x, DeRef(x) + 1)
    in 
    let double = function (x) 
               AssignRef(x, DeRef(x) * 2)
    in 
    let r = NewRef(15) in 
    let tmp0 = double(r) in 
    let tmp1 = incr(r) in 
    let tmp2 = double(r) in DeRef(r) 
IN ENVIRONMENT:
    {}
AND STORE:
~~~
<table>
    <caption>Memory Size: 0</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
</table>

##### 2
* evaluate the binding expression of the incr identifier
    * its a closure
* extend the environment to map incr to the closure
* evaluate the body expressiong of the let expression
~~~
EVALUATE:
    let double = function (x) 
               AssignRef(x, DeRef(x) * 2)
    in 
    let r = NewRef(15) in 
    let tmp0 = double(r) in 
    let tmp1 = incr(r) in 
    let tmp2 = double(r) in DeRef(r) 
IN ENVIRONMENT:
    {
        incr -> Closure(x, AssignRef(x, DeRef(x) + 1), {})
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 0</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
</table>

##### 3
* evaluate the binding expression of the double identifier
    * its a closure
    * it holds the definition of incr inside of it even though it's not neeeded
    * thanks static scoping...
    * perhaps that could be optimized in the interpreter... (something about free vars in function bodies)
* extend the environment to map double to the closure
* evaluate the body expressiong of the let expression
~~~
EVALUATE:
    let r = NewRef(15) in 
    let tmp0 = double(r) in 
    let tmp1 = incr(r) in 
    let tmp2 = double(r) in DeRef(r) 
IN ENVIRONMENT:
    {
        double -> Closure(x, AssignRef(x, DeRef(x) * 2), {incr -> Closure(x, AssignRef(x, DeRef(x) + 1), {})}),
        incr -> Closure(x, AssignRef(x, DeRef(x) + 1), {})
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 0</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
</table>

##### 4
* evaluate the binding expression of the r identifier
    * its a reference
    * we must randomly get a reference from the store
    * put 15 in that address space
    * I chose address 3, b/c why not...
* extend the environment to map r to the reference
* evaluate the body expressiong of the let expression
~~~
EVALUATE:
    let tmp0 = double(r) in 
    let tmp1 = incr(r) in 
    let tmp2 = double(r) in DeRef(r) 
IN ENVIRONMENT:
    {
        r -> Reference(0)
        double -> Closure(x, AssignRef(x, DeRef(x) * 2), {incr -> Closure(x, AssignRef(x, DeRef(x) + 1), {})}),
        incr -> Closure(x, AssignRef(x, DeRef(x) + 1), {})
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>15</td>
    </tr>
</table>

##### 5
* evaluate the binding expression of the tmp0 identifier
    * call double with r
    * double the value in the address associated with r
    * don't change r at all
* extend the environment to map tmp0 to the returned value from the function call
* evaluate the body expressiong of the let expression
~~~
EVALUATE:
    let tmp1 = incr(r) in 
    let tmp2 = double(r) in DeRef(r) 
IN ENVIRONMENT:
    {
        tmp0 -> 30,
        r -> Reference(0),
        double -> Closure(x, AssignRef(x, DeRef(x) * 2), {incr -> Closure(x, AssignRef(x, DeRef(x) + 1), {})}),
        incr -> Closure(x, AssignRef(x, DeRef(x) + 1), {})
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>30</td>
    </tr>
</table>

##### 6
* evaluate the binding expression of the tmp1 identifier
    * call incr with r
    * increment the value in the address associated with r
    * don't change r at all
* extend the environment to map tmp1 to the returned value from the function call
* evaluate the body expressiong of the let expression
~~~
EVALUATE:
    let tmp2 = double(r) in DeRef(r) 
IN ENVIRONMENT:
    {
        tmp1 -> 31,
        tmp0 -> 30,
        r -> Reference(0),
        double -> Closure(x, AssignRef(x, DeRef(x) * 2), {incr -> Closure(x, AssignRef(x, DeRef(x) + 1), {})}),
        incr -> Closure(x, AssignRef(x, DeRef(x) + 1), {})
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>31</td>
    </tr>
</table>

##### 7
* evaluate the binding expression of the tmp2 identifier
    * call double with r
    * double the value in the address associated with r
    * don't change r at all
* extend the environment to map tmp2 to the returned value from the function call
* evaluate the body expressiong of the let expression
~~~
EVALUATE:
    DeRef(r) 
IN ENVIRONMENT:
    {
        tmp2 -> 62
        tmp1 -> 31,
        tmp0 -> 30,
        r -> Reference(0),
        double -> Closure(x, AssignRef(x, DeRef(x) * 2), {incr -> Closure(x, AssignRef(x, DeRef(x) + 1), {})}),
        incr -> Closure(x, AssignRef(x, DeRef(x) + 1), {})
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>62</td>
    </tr>
</table>

##### 8
* Dereferenc r
    * r represents address 3
    * address 3 holds value 62
* return 62
~~~
EVALUATE:
    62 
IN ENVIRONMENT:
    {
        tmp2 -> 62
        tmp1 -> 31,
        tmp0 -> 30,
        r -> Reference(0),
        double -> Closure(x, AssignRef(x, DeRef(x) * 2), {incr -> Closure(x, AssignRef(x, DeRef(x) + 1), {})}),
        incr -> Closure(x, AssignRef(x, DeRef(x) + 1), {})
    }
AND STORE:
~~~
<table>
    <caption>Memory Size: 1</caption>
    <tr>
        <th>Address</th>
        <th>Value</th>
    </tr>
    <tr>
        <td>0</td>
        <td>62</td>
    </tr>
</table>

##### 9
* We've found a value!
* the value of the below lettuce program is 62:

~~~
let incr = function (x) 
           AssignRef(x, DeRef(x) + 1)
in 
let double = function (x) 
           AssignRef(x, DeRef(x) * 2)
in 
let r = NewRef(15) in 
let tmp0 = double(r) in 
let tmp1 = incr(r) in 
let tmp2 = double(r) in
  DeRef(r) 
~~~

##### Qs????

### Are we missing something?
* We can create references to values
* We can read from the reference
* We can write to the reference
* Is there anything else that we'd like?
    * How would we add that in?
    * How do other langauges handle such things?
    * Are there security concerns?

### Immutable Store
* We implement a data type that we name 'ImmutableStore' to help us get the above behavior in the interpreter 
* The store starts and length 0
    * well, it could start anywhere really, but 0 is convenient and logical
* If we add a reference, then the first address will be 0 and the length becomes 1
* ...then so on

In [ ]:
sealed trait Expr
sealed trait Value extends Expr
type Address = Int


// nCells: the current size of the store
// storeMap: mapping of addresses to values
case class ImmutableStore(val nCells: Int, val storeMap: Map[Address, Value])


// consider our AST:
case class NewRef(e: Expr) extends Expr
case class DeRef(lval: Expr) extends Expr
case class AssignRef(lval: Expr, rval: Expr) extends Expr


// Which AST node do we use this for? _______
def createNewCell(s: ImmutableStore, v: Value): (ImmutableStore, Address) = {
        /*- make a new cell -*/
        val j = s.nCells
        val nMap = s.storeMap + (j -> v)
        val nStore = ImmutableStore(s.nCells + 1, nMap) // Make a new store with one more cell
        (nStore, j)
}    

// Which AST node do we use this for? _______
def lookupCellValue(s: ImmutableStore, j: Address): Value = {
        if (s.storeMap.contains(j))b{
            s.storeMap(j)
        } else {
            throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j")
        }
}

// Which AST node do we use this for? _______
def assignToCell(s: ImmutableStore, j: Address, v: Value): ImmutableStore = {
        if (s.storeMap.contains(j)){
            val nMap = s.storeMap + (j -> v) // Update the store map.
            ImmutableStore(s.nCells, nMap)
        } else {
            throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
        }
    }

## New Eval
* Now that we have mutability represented with a store, eval changes
* Now eval has 3 parameters
    * e
    * env
    * m - store/memory/whatever you want to call it, I call it 'm' for memory because I am lazy
* consider that every expression in Lettuce could alter memory in some way
* b/c of this, now eval returns a tuple of type (Value, ImmutableStore)
* we not only get the value of the expression, but also the memory state after the expression has been evaluated
* Operational semantic
    * (v, mp) = eval(e, env, m)
* Inference rule
    * $$\begin{array} &
(n1, mp) = eval(e1, env, m)
\hspace{2cm}
(n2, mpp) = eval(e2, env, mp)
\hspace{2cm}
vp = n1 * n2
\\
\hline
(vp, mpp) = eval(e1 * e2, env, m)
\end{array} (EvalMult)$$
* Note
    * m might equal mp
    * mp might equal mpp
    * but we still need to use the most current memory state when evaluating each expression

In [ ]:
sealed trait Expr
sealed trait Value extends Expr
type Address = Int
type Number = Double
type Identifier = String

sealed trait Environment 
case object EmptyEnv extends Environment
case class Extend(x: Identifier, v: Value, sigma: Environment) extends Environment
case class ExtendRec(f: Identifier, x: Identifier, e: Expr, sigma: Environment ) extends Environment

case class Mult(e1:Expr, e2:Expr) extends Expr

case class Const(n1:Number) extends Value

// nCells: the current size of the store
// storeMap: mapping of addresses to values
case class ImmutableStore(val nCells: Int, val storeMap: Map[Address, Value])


def eval(e:Expr, env:Environment = EmptyEnv, m:ImmutableStore = ImmutableStore(0, Map())): (Value, ImmutableStore) = {
    e match {
        case v:Value => (v,m)
        case Mult(e1,e2) => ???
    }
}

assert(eval(Mult(Const(2.0), Const(3.0)))._1 == Const(6.0))

### Teaser Continuation
* There is a concept termed "CPS" or "Continuation Passing Style"
* We are planning to discuss it next week
* It helps turn things into their tail recursive versions of themselves
* pass an argument to the function that says...
* if I knew the result of this function call, this is what I would do next
* Often times I call this parameter either
    * 'sc' for success continuation
    * OR
    * 'andThen' because it says what to do next
* I can write all this using CPS

In [18]:
sealed trait Expr
sealed trait Value extends Expr
type Address = Int
type Number = Double
type Identifier = String

sealed trait Environment 
case object EmptyEnv extends Environment
case class Extend(x: Identifier, v: Value, sigma: Environment) extends Environment
case class ExtendRec(f: Identifier, x: Identifier, e: Expr, sigma: Environment ) extends Environment

case class Mult(e1:Expr, e2:Expr) extends Expr

case class Const(n1:Number) extends Value

// nCells: the current size of the store
// storeMap: mapping of addresses to values
case class ImmutableStore(val nCells: Int, val storeMap: Map[Address, Value])


def eval(e:Expr):Value = {
    
    def e2v(e:Expr, env:Environment, m:ImmutableStore)(andThen:((Value, ImmutableStore)) => (Value, ImmutableStore)): (Value, ImmutableStore) = {
        e match {
            case v:Value => ???
            case Mult(e1,e2) => applyArith(e1,e2){ _* _ }
        }
    }
    
    val (v, _) = e2v(e,EmptyEnv, ImmutableStore(0, Map()))(r => r)
    v
    
}

assert(eval(Mult(Const(2.0), Const(3.0))) == Const(6.0))
assert(eval(Mult(Mult(Const(2.0), Const(3.0)),Const(5.0))) == Const(30.0))

cmd18.sc:26: not found: value applyArith
            case Mult(e1,e2) => applyArith(e1,e2){ _* _ }
                                ^Compilation Failed

: 

## Implicit References
* We looked at explicit references for mutability in lettuce
* it's fine...
* what about implicit references?
* Consider the following

### Concrete Lettuce Grammar - implicit refs
$$\begin{array} &
e & \rightarrow & let\ var\ x\ =\ e\ in\ e \\
& | & AssignVar(e,e) \\
\\
x & \rightarrow & string \\
\\
\end{array} $$

#### Concrete Lettuce Examples

##### Easy Example
###### EXPLICIT REF
~~~
let x = NewRef(10) in 
let y = DeRef(x) + 1 in 
let z = AssignRef(x, y) in 
DeRef(x)
~~~
###### Implicit REF
~~~
let var x = 10 in
let y = x + 1 in
let z = AssignVar(x,y) in
x
~~~

##### Hard Example
###### EXPLICIT REF
~~~
let incr = function (x) AssignRef(x, DeRef(x) + 1)
in 
let double = function (x) AssignRef(x, DeRef(x) * 2)
in 
let r = NewRef(15) in 
let tmp0 = double(r) in 
let tmp1 = incr(r) in 
let tmp2 = double(r) in
DeRef(r) 
~~~
###### Implicit REF
~~~
let incr = function (x) AssignVar(x, x + 1)
in 
let double = function (x) AssignVar(x, x * 2)
in 
let var r = 15 in 
let tmp0 = double(r) in 
let tmp1 = incr(r) in 
let tmp2 = double(r) in
r
~~~

#### Comments on Concrete Lettuce
* you can decide for yourself which is a "better" syntax
* It's an aesthetic choice
* Doesn't make much of a differnce in the performance of the langauge
* It does change how the programmer needs to think while programming
* Implicit reference trade-off:
    * less syntax to remember
        * As a lettuce programmer I don't need to type DeRef and NewRef...
    * less explicit syntax, ambiguous syntax
        * 'x' might be a reference to something in our mememory/ImmutableStore
        * 'x' might be something simple and exists in the environment

### Generative Grammar for Lettuce AST - implicit refs
$$\begin{array} &
Expr & \rightarrow & LetVar(Identifier,Expr,Expr)\\
& | & AssignVar(Expr,Expr)\\
& | & Value \\
\\
Value & \rightarrow & Reference(Address) \\
\\
Identifier & \rightarrow & String \\
\\
Address & \rightarrow & Int \\
\\
\end{array} $$

### Interpreter
* basically the same, but now we have less nodes to look at
* Ident(Identifier) because somewhat special
    * it might be a simple value in the environment
    * it might be a reference
    * Our code will have to check and behave accordingly
* Sriram's notes have the code available for your review
* We'll discuss this further next week

## Solutions

### New Eval

In [5]:
sealed trait Expr
sealed trait Value extends Expr
type Address = Int
type Number = Double
type Identifier = String

sealed trait Environment 
case object EmptyEnv extends Environment
case class Extend(x: Identifier, v: Value, sigma: Environment) extends Environment
case class ExtendRec(f: Identifier, x: Identifier, e: Expr, sigma: Environment ) extends Environment

case class Mult(e1:Expr, e2:Expr) extends Expr

case class Const(n1:Number) extends Value

// nCells: the current size of the store
// storeMap: mapping of addresses to values
case class ImmutableStore(val nCells: Int, val storeMap: Map[Address, Value])


def eval(e:Expr, env:Environment = EmptyEnv, m:ImmutableStore = ImmutableStore(0, Map())): (Value, ImmutableStore) = {
    e match {
        case v:Value => (v,m)
        case Mult(e1,e2) => {
            val (v1, mp) = eval(e1,env,m)
            val (v2, mpp) = eval(e2, env, mp)
            (v1,v2) match {
                case (Const(n1),Const(n2)) => {
                    val np = n1 * n2
                    val vp = Const(np)
                    println(np)
                    (vp, mpp)
                }
                case _ => ???
            }
        }
    }
}

assert(eval(Mult(Const(2.0), Const(3.0)))._1 == Const(6.0))

6.0


defined trait Expr
defined trait Value
defined type Address
defined type Number
defined type Identifier
defined trait Environment
defined object EmptyEnv
defined class Extend
defined class ExtendRec
defined class Mult
defined class Const
defined class ImmutableStore
defined function eval

### New Eval with Continuations

In [18]:
sealed trait Expr
sealed trait Value extends Expr
type Address = Int
type Number = Double
type Identifier = String

sealed trait Environment 
case object EmptyEnv extends Environment
case class Extend(x: Identifier, v: Value, sigma: Environment) extends Environment
case class ExtendRec(f: Identifier, x: Identifier, e: Expr, sigma: Environment ) extends Environment

case class Mult(e1:Expr, e2:Expr) extends Expr

case class Const(n1:Number) extends Value

// nCells: the current size of the store
// storeMap: mapping of addresses to values
case class ImmutableStore(val nCells: Int, val storeMap: Map[Address, Value])


def eval(e:Expr):Value = {
    
    def e2v(e:Expr, env:Environment, m:ImmutableStore, andThen:((Value, ImmutableStore)) => (Value, ImmutableStore)): (Value, ImmutableStore) = {
        
        def applyArith(e1:Expr, e2:Expr)(f:(Number, Number) => Number):(Value,ImmutableStore) = {
            e2v(e1,env,m, {
                case (Const(n1),mp) => e2v(e2,env,mp, {
                    case (Const(n2),mpp) => andThen((Const(f(n1,n2)), mpp))
                    case _ => throw new IllegalArgumentException(s"e2 not a number in $e")
                })
                case _ => throw new IllegalArgumentException(s"e1 not a number in $e")
            })
        }
        
        e match {
            case v:Value => andThen((v,m))
            case Mult(e1,e2) => applyArith(e1,e2){_*_}
        }
        
    }
    
    val (v, _) = e2v(e,EmptyEnv, ImmutableStore(0, Map()), r => r)
    v
}

assert(eval(Mult(Const(2.0), Const(3.0))) == Const(6.0))
assert(eval(Mult(Mult(Const(2.0), Const(3.0)),Const(5.0))) == Const(30.0))

defined trait Expr
defined trait Value
defined type Address
defined type Number
defined type Identifier
defined trait Environment
defined object EmptyEnv
defined class Extend
defined class ExtendRec
defined class Mult
defined class Const
defined class ImmutableStore
defined function eval

### Just enough eval for hard example

In [13]:
/** AST **/
sealed trait Program
sealed trait Expr
sealed trait Value extends Expr
type Address = Int
type Identifier = String
type Number = Double

sealed trait Environment 
case object EmptyEnv extends Environment
case class Extend(x: Identifier, v: Value, sigma: Environment) extends Environment
case class ExtendRec(f: Identifier, x: Identifier, e: Expr, sigma: Environment ) extends Environment

case class TopLevel(e:Expr) extends Program

case class Mult(e1:Expr, e2:Expr) extends Expr
case class Add(e1:Expr, e2:Expr) extends Expr
case class FunDef(x:Identifier, e:Expr) extends Expr
case class FunCall(e1:Expr,e2:Expr) extends Expr
case class NewRef(e: Expr) extends Expr
case class DeRef(lval: Expr) extends Expr
case class AssignRef(lval: Expr, rval: Expr) extends Expr
case class Let(x:Identifier, e1:Expr, e2:Expr) extends Expr
case class Ident(x:Identifier) extends Expr

case class Const(n:Number) extends Value
case class Reference(a:Address) extends Value
case class Closure(x:Identifier, e:Expr, env:Environment) extends Value

/*-- Operations on environments --*/

def lookupEnv(env: Environment, x: Identifier): Value = env match {
    case EmptyEnv => throw new IllegalArgumentException(s"Error could not find string $x in environment")
    case Extend(y, v, _) if y == x => v
    case Extend(_, _, envp) => lookupEnv(envp, x)
    case ExtendRec(f, y, e, envp) => if (x == f) 
                                          Closure(y, e, env)
                                   else
                                          lookupEnv(envp, x)
}

/** ImmutableStore **/

// nCells: the current size of the store
// storeMap: mapping of addresses to values
case class ImmutableStore(val nCells: Int, val storeMap: Map[Address, Value])


/** Operation on ImmutableStore **/

// Used in eval for ???
def createNewCell(s: ImmutableStore, v: Value): (ImmutableStore, Address) = {
        /*- make a new cell -*/
        val j = s.nCells
        val nMap = s.storeMap + (j -> v)
        val nStore = ImmutableStore(s.nCells + 1, nMap) // Make a new store with one more cell
        (nStore, j)
}    

// Used in eval for ???
def lookupCellValue(s: ImmutableStore, j: Address): Value = {
        if (s.storeMap.contains(j)){
            s.storeMap(j)
        } else {
            throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j")
        }
}

// Used in eval for ???
def assignToCell(s: ImmutableStore, j: Address, v: Value): ImmutableStore = {
        if (s.storeMap.contains(j)){
            val nMap = s.storeMap + (j -> v) // Update the store map.
            ImmutableStore(s.nCells, nMap)
        } else {
            throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
        }
    }

/** Interpreter **/
def evalProgram(p:Program):Value = {
    val TopLevel(e) = p
    // static checks...
    val (v,_) = eval(e)
    v
}

def eval(e:Expr, env:Environment = EmptyEnv, m:ImmutableStore = ImmutableStore(0, Map())): (Value, ImmutableStore) = {

    def e2v(e:Expr):(Value, ImmutableStore) = eval(e, env, m) 
    
    def applyBinArith(e1:Expr, e2:Expr)(f:(Number,Number) => Number):(Value, ImmutableStore) = {
        e2v(e1) match {
            case (Const(n1),mp) => eval(e2, env, mp) match {
                case (Const(n2),mpp) => (Const(f(n1,n2)),mpp)
                case _ => ???
            }
            case _ => ???
        }
    }
        
    e match {
        case v:Value => (v, m)
        case Add(e1,e2) => applyBinArith(e1,e2){_+_}
        case Mult(e1,e2) => applyBinArith(e1,e2){_*_}
        case FunDef(x,eBody) => (Closure(x,eBody,env), m)
        case Let(x,e1,e2) => {
            val (v2,mp) = e2v(e1)
            eval(e2, Extend(x,v2,env), mp)
        }
        case Ident(x) => (lookupEnv(env, x), m)
        case FunCall(e1,arg) => e2v(e1) match {
            case (Closure(x,eBody,envOld),mp) => {
                val (v2, mpp) = eval(arg,env,mp)
                eval(eBody, Extend(x,v2,envOld), mpp)
            }
            case _ => ???
        }
        
        // New stuff:
        case NewRef(e) => {
            val (v, mp) = e2v(e)
            val (mpp, a) = createNewCell(mp, v)
            (Reference(a), mpp)
        }
        case DeRef(e1) => e2v(e1) match {
            case (Reference(a), mp) => (lookupCellValue(mp, a), mp)
            case _ => ???
        }
        case AssignRef(e1, e2) => {
            e2v(e1) match {
                case (Reference(a),mp) => {
                    val (v2, mpp) = eval(e2, env, mp)
                    val mppp = assignToCell(mpp,a,v2)
                    (v2, mppp)
                }
                case _ => ???
            }
        }
    }
}

// let incr = function (x) 
//            AssignRef(x, DeRef(x) + 1)
// in 
// let double = function (x) 
//            AssignRef(x, DeRef(x) * 2)
// in 
// let r = NewRef(15) in 
// let tmp0 = double(r) in 
// let tmp1 = incr(r) in 
// let tmp2 = double(r) in
//   DeRef(r)
val p = TopLevel(Let(
    "incr",
    FunDef("x",AssignRef(Ident("x"), Add(DeRef(Ident("x")),Const(1.0)))),
    Let(
        "double",
        FunDef("x",AssignRef(Ident("x"), Mult(DeRef(Ident("x")),Const(2.0)))),
        Let(
            "r",
            NewRef(Const(15.0)),
            Let(
                "tmp0",
                FunCall(Ident("double"),Ident("r")),
                Let(
                    "tmp1",
                    FunCall(Ident("incr"),Ident("r")),
                    Let(
                        "tmp2",
                        FunCall(Ident("double"),Ident("r")),
                        DeRef(Ident("r"))
                    )
                )
            )
        )
    )
))

val v = Const(62.0)

assert(evalProgram(p) == v)
println("\n\n\n\nPASSED ALL TESTS!!!!!\n\n\n")





PASSED ALL TESTS!!!!!





defined trait Program
defined trait Expr
defined trait Value
defined type Address
defined type Identifier
defined type Number
defined trait Environment
defined object EmptyEnv
defined class Extend
defined class ExtendRec
defined class TopLevel
defined class Mult
defined class Add
defined class FunDef
defined class FunCall
defined class NewRef
defined class DeRef
defined class AssignRef
defined class Let
defined class Ident
defined class Const
defined class Reference
defined class Closure
defined function lookupEnv
defined class ImmutableStore
defined function createNewCell
defined function lookupCellValue
defined function assignToCell
defined function evalProgram
defined function eval
p: TopLevel = TopLevel(
  Let(
    "incr",
    FunDef("x", AssignRef(Ident("x"), Add(DeRef(Ident("x")), Const(1.0)))),
    Let(
      "double",
      FunDef("x", AssignRef(Ident("x"), Mult(DeRef(Ident("x")), Const(2.0)))),
      Let(
        "r",
        NewRef(Const(15.0)),
        Let(
          "tmp0",

## Overview
* Explicit References
* New Eval
* Implicit References

## TODOs:
* Homework and Quiz are going live soon - they will be due next Friday
* Project 2 will go live this weekend
    * It will be due after Spring Break
    * It will create Lettuce with Figs
    * Or rather, a variant of lettuce that works with Figures
        * lines
        * circles
        * squares
        * and so on
    * it can be used to create neat fractle patterns
    * with a few extentions it could be used to create a tool similar to power point
* AST visuallizer posted to Piazza
    * a student in Fall'18 created a visualizer for ASTs
    * highly recommend that you check this out
* LGBTQ-friendly Employer & Student Mixer
    * Tonight 5:30-7:00pm
    * UMC Aspen Room
    * Appetizers served.
    * Free for all CU Boulder students! No need to sign up - just show up.